In [1]:
# import libraries
from typing import List, Union

try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

2.19.0-dev20241127


In [2]:
# get data files
!wget -q https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget -q https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [3]:
# Check for headers
!head -1 train-data.tsv; head -1 valid-data.tsv

ham	ahhhh...just woken up!had a bad dream about u tho,so i dont like u right now :) i didnt know anything about comedy night but i guess im up for it.
ham	i am in hospital da. . i will return home in evening


In [4]:
# Load the datasets
df_train = pd.read_csv(train_file_path, sep="\t", names=["label", "text"])
df_test = pd.read_csv(test_file_path, sep="\t", names=["label", "text"])

In [5]:
print(df_train.shape, df_test.shape)

(4179, 2) (1392, 2)


In [6]:
# Encode labels
df_train["label"] = df_train["label"].map({"ham": 0, "spam": 1})
df_test["label"] = df_test["label"].map({"ham": 0, "spam": 1})

In [7]:
df_train.tail()

,label,text
4174,0,just woke up. yeesh its late. but i didn't fal...
4175,0,what do u reckon as need 2 arrange transport i...
4176,1,free entry into our £250 weekly competition ju...
4177,1,-pls stop bootydelious (32/f) is inviting you ...
4178,0,tell my bad character which u dnt lik in me. ...


In [8]:
# Separate texts and labels
X_train = df_train["text"].values
y_train = df_train["label"].values

X_test = df_test["text"].values
y_test = df_test["label"].values

In [9]:
# Tokenize the text data
num_words = 15_000
tokenizer = keras.preprocessing.text.Tokenizer(
    num_words=num_words, oov_token="<OOV>"
)
tokenizer.fit_on_texts(X_train)

# Convert texts to sequences
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

In [10]:
# Get the max message length using the 99th percentile
max_length = int(
    np.percentile(df_train["text"].apply(lambda x: len(x.split())), 99)
)

# Pad sequences to ensure uniform input length
X_train_padded = keras.preprocessing.sequence.pad_sequences(
    X_train_seq, maxlen=max_length, padding="post", truncating="post"
)
X_test_padded = keras.preprocessing.sequence.pad_sequences(
    X_test_seq, maxlen=max_length, padding="post", truncating="post"
)

In [11]:
# Build model
model = keras.models.Sequential([
    keras.layers.Embedding(input_dim=num_words, output_dim=128),
    keras.layers.LSTM(64, return_sequences=False),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(64, activation="relu"),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(1, activation="sigmoid"),
])

In [12]:
# Compile the model
model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"],
)
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [13]:
# Train the model
model.fit(
    X_train_padded,
    y_train,
    epochs=10,
    batch_size=32,
    validation_data=(X_test_padded, y_test)
)

AttributeError: module 'ml_dtypes' has no attribute 'float8_e3m4'
Epoch 1/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 18s 86ms/step - accuracy: 0.8411 - loss: 0.4654 - val_accuracy: 0.8657 - val_loss: 0.4008
Epoch 2/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 8s 63ms/step - accuracy: 0.8729 - loss: 0.3816 - val_accuracy: 0.9461 - val_loss: 0.1504
Epoch 3/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 9s 67ms/step - accuracy: 0.8781 - loss: 0.4058 - val_accuracy: 0.8657 - val_loss: 0.3957
Epoch 4/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 7s 55ms/step - accuracy: 0.8774 - loss: 0.3802 - val_accuracy: 0.8657 - val_loss: 0.3907
Epoch 5/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 9s 69ms/step - accuracy: 0.8927 - loss: 0.2808 - val_accuracy: 0.9569 - val_loss: 0.2016
Epoch 6/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 9s 66ms/step - accuracy: 0.9787 - loss: 0.0923 - val_accuracy: 0.9871 - val_loss: 0.0514
Epoch 7/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 7s 56ms/step - accuracy: 0.9949 - loss: 0.0255 - val_accuracy: 0.9871 - val_loss: 0.0538
Epoch 8/10
131/131 ━━━━━━━━━━━

In [14]:
def predict_message(pred_text: str) -> List[Union[float, str]]:
    # Tokenize the text
    pred_seq = tokenizer.texts_to_sequences([pred_text])

    # Pad the sequence
    pred_padded = keras.preprocessing.sequence.pad_sequences(
    pred_seq, maxlen=max_length, padding="post", truncating="post"
    )

    # Get the prediction from the model
    prediction_prob = model.predict(pred_padded)[0][0]

    # Convert the probability to a label
    prediction = "spam" if prediction_prob >= 0.5 else "ham"

    return [prediction_prob, prediction]


texts = [
  "what's going on? did you get that science fair prize?",
  "sign up for our newsletter and claim $100 in BTC!",
]

for text in texts:
  print(predict_message(text))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 211ms/step
[0.00024222361, 'ham']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
[0.98519534, 'spam']


In [15]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
You passed the challenge. Great job!
